In [1]:
import os 
import numpy as np
from pathlib import Path
import plotly.express as px
import pandas as pd

In [2]:
meticDF = pd.read_csv("results_2_2-16.csv")
boms = meticDF["BOM"].unique()
bom_nodes={}
for bom,node in meticDF[["BOM","Nodes"]].drop_duplicates().values.tolist():
    bom_nodes[bom]=node
bom_nodes

{'bom_2_2_10_5': 7,
 'bom_2_3_10_5': 13,
 'bom_2_4_10_5': 21,
 'bom_2_5_10_5': 31,
 'bom_2_6_10_5': 43,
 'bom_2_7_10_5': 57,
 'bom_2_8_10_5': 73,
 'bom_2_9_10_5': 91,
 'bom_2_10_10_5': 111,
 'bom_2_11_10_5': 133,
 'bom_2_12_10_5': 157,
 'bom_2_13_10_5': 183,
 'bom_2_14_10_5': 211,
 'bom_2_15_10_5': 241,
 'bom_2_16_10_5': 273}

In [3]:
for file in ["results_2_2-16_LS_3min.csv","results_2_2-16_BS_3min.csv","results_2_2-16_BS_6min.csv","results_2_2-16_BS_60min.csv"]:
    df=pd.read_csv(file)
    for bom in boms:
        for alg in list(df['Algorithm'].unique()):
            for op in ["OFF", "ON"]:
                row=[bom, bom_nodes[bom], f"{alg}{'_opt' if op=='ON' else ''}"]
                sub_df = df[(df['Bom']==bom) & (df['Algorithm']==alg) &(df['Optimizations']==op)]['Makespan'].to_list()
                if len(sub_df) > 0:
                    row.append(np.round(np.mean(sub_df),3))
                    row.append(np.round(np.std(sub_df),3))
                    row.append(np.min(sub_df))
                    row.append(np.max(sub_df))
                    #print(row)
                    meticDF.loc[len(meticDF)] = row

meticDF.to_csv("benchmark_results_2_2-16.csv", index=False)

In [6]:
fig = px.line(meticDF, x="Nodes", y="Makespan", error_y="std_Makespan", color="Algorithm", color_discrete_sequence=px.colors.qualitative.Light24,height=600)
fig.show()
fig.write_html("benchmark.html", include_plotlyjs=True)

In [5]:
from scipy import stats

opt = "ON"
df = pd.read_csv("results_2_2-16_BS_3min.csv")
f_res = pd.DataFrame(columns=['test', 'statistic', 'pvalue', 'difference'])

for bom in boms:
    abc = df[(df['Bom'] == bom) & (df['Algorithm'] == "BS_ABC_3min") & (df['Optimizations'] == opt)]['Makespan'].to_list()
    ga = df[(df['Bom'] == bom) & (df['Algorithm'] == "BS_GA_3min") & (df['Optimizations'] == opt)]['Makespan'].to_list()
    ts = df[(df['Bom'] == bom) & (df['Algorithm'] == "BS_TS_3min") & (df['Optimizations'] == opt)]['Makespan'].to_list()
    #print(bom, abc, ga, ts, sep='\n')
    f_test = stats.friedmanchisquare(abc, ga, ts)

    f_res.loc[len(f_res)] = [bom, f_test[0], f_test[1], True if f_test[1] < 0.05 else False]

f_res


,test,statistic,pvalue,difference
0,bom_2_2_10_5,52.811881,3.404440e-12,True
1,bom_2_3_10_5,55.364706,9.499634e-13,True
2,bom_2_4_10_5,52.907407,3.245656e-12,True
3,bom_2_5_10_5,58.066667,2.460274e-13,True
4,bom_2_6_10_5,50.400000,1.137049e-11,True
5,bom_2_7_10_5,45.600000,1.253389e-10,True
6,bom_2_8_10_5,54.600000,1.392389e-12,True
7,bom_2_9_10_5,60.000000,9.357623e-14,True
8,bom_2_10_10_5,45.600000,1.253389e-10,True
9,bom_2_11_10_5,56.266667,6.051298e-13,True


In [26]:
meticDF = pd.read_csv("benchmark2.csv")
fig = px.line(meticDF[meticDF['Algorithm'].isin(["LETSA", "CPLEX 1h", "Genetic Algorithm 3min", 'Tabu Search 3min'])], x="Nodes", y="Makespan", error_y="std_Makespan", color="Algorithm")
fig.show()
fig.write_html("benchmark_scalabilitate.html", include_plotlyjs=True)